In [2]:
import os
import re
import openai
from tqdm.auto import tqdm
from dotenv import load_dotenv
from tkinter import Tk, filedialog
from termcolor import colored
from IPython.display import display, HTML

# Load environment variables from .env file
load_dotenv()

# Retrieve the OpenAI API key from environment variables
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
if not OPENAI_API_KEY:
    raise ValueError("API key not found. Set the OPENAI_API_KEY environment variable in your .env file.")

# Initialize OpenAI client
openai.api_key = OPENAI_API_KEY

def translate_text(text, source_lang="en", target_lang="pt-PT"):
    """Translate text using GPT-4-turbo."""
    # Display the text being translated
    # display(Markdown(f"**Translating text:** `{text[:100]}...`"))  # Uncomment if needed for debugging
    response = openai.chat.completions.create(
        model="gpt-4-turbo",
        messages=[
            {"role": "system", "content": f"You are a translator. Translate from {source_lang} to {target_lang}."},
            {"role": "user", "content": text}
        ],
        temperature=0,
        max_tokens=500
    )
    # Extract the translation from the response
    translation = response.choices[0].message.content.strip()
    return translation

def parse_srt(file_path):
    """Parse SRT file into a list of tuples (index, timestamp, text)."""
    with open(file_path, 'r', encoding='utf-8') as file:
        content = file.read()
    
    subtitles = []
    parts = re.split(r'\n\n', content.strip())
    for part in parts:
        lines = part.split('\n')
        if len(lines) >= 3:
            index = lines[0]
            timestamp = lines[1]
            text = '\n'.join(lines[2:])
            subtitles.append((index, timestamp, text))
    return subtitles

def write_srt(subtitles, file_path):
    """Write translated subtitles to a new SRT file."""
    with open(file_path, 'w', encoding='utf-8') as file:
        for index, timestamp, text in subtitles:
            file.write(f"{index}\n{timestamp}\n{text}\n\n")

def display_translations(original, translated):
    """Display original and translated texts in colored HTML."""
    html_content = f"""
    <div style='color: blue; font-weight: bold;'>Original: {original}</div>
    <div style='color: green; font-weight: bold;'>Translated: {translated}</div>
    """
    display(HTML(html_content))

def main(input_file):
    print(colored(f"Reading file: {input_file}", "cyan"))
    subtitles = parse_srt(input_file)
    
    # Generate the output file name
    base, ext = os.path.splitext(input_file)
    output_file = f"{base}_PT-PT{ext}"
    
    # Translate subtitles with a progress bar
    translated_subtitles = []
    for index, timestamp, text in tqdm(subtitles, desc="Translating Subtitles"):
        # Skip sounds or comments
        if re.search(r'\[.*?\]', text):
            translated_subtitles.append((index, timestamp, text))
            continue
        
        # Translate and correct the text
        translated_text = translate_text(text)
        
        # Ensure text fits in 2 lines
        lines = translated_text.split('\n')
        if len(lines) > 2:
            translated_text = '\n'.join(lines[:2])
        
        # Display original and translated texts in colored HTML
        display_translations(text, translated_text)
        
        translated_subtitles.append((index, timestamp, translated_text))
    
    # Write the translated subtitles to a new SRT file
    write_srt(translated_subtitles, output_file)
    print(colored(f"Translation complete! Translated file saved as {output_file}", "green"))

if __name__ == "__main__":
    # Create a Tkinter root window and hide it
    root = Tk()
    root.withdraw()
    
    # Ask the user to select an SRT file
    input_subtitle_path = filedialog.askopenfilename(
        title="Select the English SRT file",
        filetypes=[("SRT files", "*.srt"), ("All files", "*.*")]
    )
    
    # Check if a file was selected
    if input_subtitle_path:
        main(input_subtitle_path)
    else:
        print(colored("No file selected. Exiting.", "red"))


Reading file: /Users/f.nuno/Downloads/Extracted_Subtitles/subtitle_track_2.srt


Translating Subtitles:   0%|          | 0/1380 [00:00<?, ?it/s]

Translation complete! Translated file saved as /Users/f.nuno/Downloads/Extracted_Subtitles/subtitle_track_2_PT-PT.srt
